In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
import os
import tensorflow as tf

import numpy as np
import libspn as spn

from deepsm.graphspn.spn_model import SpnModel, mod_compute_graph_up
from deepsm.graphspn.tbm.spn_template import TemplateSpn, NodeTemplateSpn, EdgeTemplateSpn
from deepsm.graphspn.tbm.spn_instance import NodeTemplateInstanceSpn
from deepsm.graphspn.tbm.template import EdgeTemplate, NodeTemplate, StarTemplate, SingletonTemplate, ThreeNodeTemplate, PairTemplate
from deepsm.graphspn.tbm.graph_builder import build_graph
from deepsm.util import CategoryManager

In [3]:
dirpath = "/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/CustomGraphExperiment/models"
star_tmpl_path = os.path.join(dirpath, "StarTemplate_%d_Freiburg-Saarbrucken-Stockholm.spn" % CategoryManager.NUM_CATEGORIES)
three_tmpl_path = os.path.join(dirpath, "ThreeNodeTemplate_%d_Freiburg-Saarbrucken-Stockholm.spn" % CategoryManager.NUM_CATEGORIES)
pair_tmpl_path = os.path.join(dirpath, "PairTemplate_%d_Freiburg-Saarbrucken-Stockholm.spn" % CategoryManager.NUM_CATEGORIES)
single_tmpl_path = os.path.join(dirpath, "SingletonTemplate_%d_Freiburg-Saarbrucken-Stockholm.spn" % CategoryManager.NUM_CATEGORIES)
graph_file = "/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/analysis/stockholm_floor7_cloudy_b/3nodes.ug"

In [4]:
topo_map, likelihoods = build_graph(graph_file)
print(len(topo_map.nodes))
print(CategoryManager.NUM_CATEGORIES)

3
2


In [5]:
sess = tf.Session()

In [6]:
#star_tmpl_spn = NodeTemplateSpn(StarTemplate, seed=100)
#star_tmpl_spn.load(star_tmpl_path, sess)
three_tmpl_spn = NodeTemplateSpn(ThreeNodeTemplate, seed=100)
three_tmpl_spn.load(three_tmpl_path, sess)
pair_tmpl_spn = NodeTemplateSpn(PairTemplate, seed=100)
pair_tmpl_spn.load(pair_tmpl_path, sess)
single_tmpl_spn = NodeTemplateSpn(SingletonTemplate, seed=100)
single_tmpl_spn.load(single_tmpl_path, sess)
SpnModel.make_weights_same(sess, single_tmpl_spn.root)

[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/CustomGraphExperiment/models/ThreeNodeTemplate_2_Freiburg-Saarbrucken-Stockholm.spn'
[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/CustomGraphExperiment/models/PairTemplate_2_Freiburg-Saarbrucken-Stockholm.spn'
[INFO] [spn.JSONLoader:load] Loading SPN graph from file '/home/zkytony/Documents/thesis/experiments/deep-semantic-mapping/deepsm/experiments/results/graphspn/CustomGraphExperiment/models/SingletonTemplate_2_Freiburg-Saarbrucken-Stockholm.spn'


Generating SPN structure...
[Using seed 100]
Generating SPN structure...
[Using seed 100]
Generating SPN structure...
[Using seed 100]


In [10]:
SpnModel.make_weights_same(sess, single_tmpl_spn.root)
#star_tmpl_spn._conc_inputs.set_inputs()
single_tmpl_spn._conc_inputs.set_inputs()
three_tmpl_spn._conc_inputs.set_inputs()
pair_tmpl_spn._conc_inputs.set_inputs()
# spn_tmpls = [(star_tmpl_spn, StarTemplate), (single_tmpl_spn, SingletonTemplate),
#               (three_tmpl_spn, ThreeNodeTemplate), (pair_tmpl_spn, PairTemplate)]
spn_tmpls2 = [(single_tmpl_spn, SingletonTemplate),
              (three_tmpl_spn, ThreeNodeTemplate), (pair_tmpl_spn, PairTemplate)]

In [11]:
instance_spn = NodeTemplateInstanceSpn(topo_map, sess, *spn_tmpls2, num_partitions=2,
                                        seq_id="floor7_cloudy_b", divisions=8,
                                        visualize_partitions_dirpath=None, db_name="Stockholm")

Partitioning the graph... (Selecting 2 from 2 attempts)
1.000  1.000  
Building instance spn...
Partition 1
Will duplicate ThreeNodeTemplate 1 times.
Duplicating... 1
Will duplicate PairTemplate 0 times.
Will duplicate SingletonTemplate 0 times.
Partition 2
Will duplicate ThreeNodeTemplate 1 times.
Duplicating... 1
Will duplicate PairTemplate 0 times.
Will duplicate SingletonTemplate 0 times.


In [12]:
instance_spn.expand()
            
print("Initializing Ops. Will take a while...")
instance_spn.init_ops(no_mpe=True)

Expanding...
Initializing Ops. Will take a while...
Initializing learning Ops...
Instructions for updating:
Use the `axis` argument instead


[WARNING] [tensorflow:warning] From /home/zkytony/Documents/thesis/experiments/libspn/libspn/graph/sum.py:298: calling argmax (from tensorflow.python.ops.math_ops) with dimension is deprecated and will be removed in a future version.
Instructions for updating:
Use the `axis` argument instead


In [13]:
spn.display_spn_graph(instance_spn._root, skip_params=False)

In [14]:
instance_spn._root

Sum_6

In [15]:
SpnModel.print_weights(instance_spn._root, sess)

Sum_6 (Sum)
[ 0.5  0.5] [(2,)]
Sum_2 (Sum)
[ 0.01057933  0.60179216  0.37704921  0.01057933] [(4,)]
Sum_5 (Sum)
[ 0.01057933  0.60179216  0.37704921  0.01057933] [(4,)]
Sum_1 (Sum)
[ 0.01669902  0.02068494  0.02068494  0.94193113] [(4,)]
Sum (Sum)
[ 0.90537304  0.03437359  0.03437359  0.0258798 ] [(4,)]
Sum_4 (Sum)
[ 0.01669902  0.02068494  0.02068494  0.94193113] [(4,)]
Sum_3 (Sum)
[ 0.90537304  0.03437359  0.03437359  0.0258798 ] [(4,)]


In [16]:
query_nids = list(topo_map.nodes.keys())
query = {k:-1 for k in query_nids}
query_lh = likelihoods

In [17]:
marginals = instance_spn.marginal_inference(sess, query_nids,
                                            query, query_lh=query_lh)

In [18]:
marginals

{159: [array([-1494.11633301], dtype=float32),
  array([-1417.60070801], dtype=float32)],
 160: [array([-1417.60070801], dtype=float32),
  array([-1541.4876709], dtype=float32)],
 162: [array([-1480.84472656], dtype=float32),
  array([-1417.60070801], dtype=float32)]}

In [19]:
SpnModel.print_structure(instance_spn._root, sess)

Sum_6 (Sum)
    Input(Product_8, None) (Product)
    Input(Product_17, None) (Product)
Product_8 (Product)
    Input(Sum_2, None) (Sum)
Product_17 (Product)
    Input(Sum_5, None) (Sum)
Sum_2 (Sum)
    Input(Product_7, None) (Product)
    Input(Product_6, None) (Product)
    Input(Product_5, None) (Product)
    Input(Product_4, None) (Product)
Sum_5 (Sum)
    Input(Product_16, None) (Product)
    Input(Product_15, None) (Product)
    Input(Product_14, None) (Product)
    Input(Product_13, None) (Product)
Product_7 (Product)
    Input(Concat, [0]) (Concat)
    Input(Sum_1, None) (Sum)
Product_6 (Product)
    Input(Concat, [1]) (Concat)
    Input(Sum_1, None) (Sum)
Product_5 (Product)
    Input(Concat, [0]) (Concat)
    Input(Sum, None) (Sum)
Product_4 (Product)
    Input(Concat, [1]) (Concat)
    Input(Sum, None) (Sum)
Product_16 (Product)
    Input(Concat, [0]) (Concat)
    Input(Sum_4, None) (Sum)
Product_15 (Product)
    Input(Concat, [1]) (Concat)
    Input(Sum_4, None) (Sum)
Produc

In [20]:
instance_spn._conc_inputs.inputs

(Input(Product_18, None),
 Input(Product_19, None),
 Input(Product_20, None),
 Input(Product_21, None),
 Input(Product_22, None),
 Input(Product_23, None))

In [21]:
likelihoods

{159: [-557.3182373, -481.20571899],
 160: [-513.4407959, -640.68658447],
 162: [-481.86907959, -419.02822876]}

In [22]:
instance_spn._label_node_map

{0: 160, 1: 162, 2: 159}

In [34]:
catg_inputs = spn.IVs(num_vars=20, num_vals=CategoryManager.NUM_CATEGORIES, name="Catg")
conc = spn.Concat(catg_inputs)
three_tmpl_spn._conc_inputs.set_inputs()
copied_tspn_root = mod_compute_graph_up(three_tmpl_spn._root,
                                        TemplateSpn._dup_fun_up, tmpl_num_vars=[3], tmpl_num_vals=[CategoryManager.NUM_CATEGORIES],
                                        conc=conc,
                                        labels=[[0,2,4]])